In [ ]:
import paho.mqtt.client as mqtt
import logging
import json
from appJar import gui
from stmpy import Machine, Driver

# TODO: choose proper MQTT broker address
MQTT_BROKER = 'mqtt.item.ntnu.no'
MQTT_PORT = 1883

# TODO: choose proper topics for communication
MQTT_TOPIC_INPUT = 'ttm4115/team_1/collar'
MQTT_TOPIC_OUTPUT = 'ttm4115/team_1/server'

"""
cat_list= ["Kitty","Mons","Roger","Purr"]
tracked_cat= ""
"""
class CollarComponent:

    def on_connect(self, client, userdata, flags, rc):
        # we just log that we are connected
        self._logger.debug('MQTT connected to {}'.format(client))

    def on_message(self, client, userdata, msg):
        self._logger.debug('Incoming message to topic {}'.format(msg.topic))
        try:
            payload = json.loads(msg.payload.decode("utf-8"))
        except Exception as err:
            self._logger.error('Message not able to unwrap')
            return
        
        # extract command
        command=payload.get('command')
        print(payload)
        if self.tracked_cat==payload.get('name'):
            if command == "cat_in":
                print(payload.get('name') + " cat is home")
                self.stm.send('cat_in')
            if command == "cat_inside_fence":
                print(payload.get('name') + " cat is inside fence")
                self.stm.send('inside_fence')
            if command == "cat_outside_fence":
                print(payload.get('name') + " cat is outside fence")
                self.stm.send('outside_fence')
            if command == "start_tracking":
                print(payload.get('name') + " cat is tracked")
                self.stm.send('start_tracking')
        else:
            print("Not tracked cat, ignoring command")
    
    def publish_command(self, command):
        payload = json.dumps(command)
        self._logger.info(command)
        self.mqtt_client.publish(MQTT_TOPIC_OUTPUT, payload=payload, qos=2)


    def __init__(self):
        # get the logger object for the component
        self._logger = logging.getLogger(__name__)
        print('logging under name {}.'.format(__name__))
        self._logger.info('Starting Component')

        # create a new MQTT client
        self._logger.debug('Connecting to MQTT broker {} at port {}'.format(MQTT_BROKER, MQTT_PORT))
        self.mqtt_client = mqtt.Client()
        # callback methods
        self.mqtt_client.on_connect = self.on_connect
        self.mqtt_client.on_message = self.on_message
        # Connect to the broker
        self.mqtt_client.connect(MQTT_BROKER, MQTT_PORT)
        # Subscribe to proper topic
        self.mqtt_client.subscribe(MQTT_TOPIC_INPUT)
        # start the internal loop to process MQTT messages
        self.mqtt_client.loop_start()
        self.create_gui()
        self.lat=0
        self.long=0

        
        
    def create_gui(self):
        self.app = gui("Collar GUI")
        
        """CARL
        def on_button_pressed(button):
            self.stm.send('start')
            self.tracked_cat= self.app.getRadioButton("Gato")
            print('Tracked cat is {} '.format(self.tracked_cat))
            self.app.showSubWindow("Cat")
        """
        ###OLE START-----------------------
        def on_btn_update_pressed():
            self.lat=self.app.getEntry("Lat")
            self.long=self.app.getEntry("Long")
            
        def on_btn_start_pressed():
            self.driver = Driver()
            self.stm = Machine(name='stm_collar', 
                                 transitions=[t0, t_start, t1, t2, t3, t4, t5, t6, t7, t8, t9], 
                                 obj=self, 
                                 states=[idle, low_freq, norm_freq, high_freq])
            self.driver.add_machine(self.stm)
            self.driver.start()
            self.tracked_cat=self.app.getEntry("Name")
            #self.rfid=self.app.getEntry("Rfid")
            self.app.hide()
            self.app.showSubWindow("Cat")
            self.stm.send("start")
            print("Should start now")
            
        def on_close(self):
            self.app.close
            
        ###OLE END-------------------------
            
        #Main Window
        """CARL
        for i in cat_list:
            self.app.addRadioButton("Gato", '{}'.format(i))
        self.app.addButton('Choose Cat', on_button_start_pressed)
        """
    
        #OLE START-------------------------
        #self.app.startSubWindow("Start")
        self.app.setBg("lightsalmon")
        self.app.setSize("200x200")
        self.app.addLabelEntry("Name")
        #self.app.addLabelEntry("Rfid")
        self.app.addButton("Start Collar",on_btn_start_pressed)
        #self.app.stopSubWindow()
        #END OLE---------------------
        
        #Pop up window
        self.app.setBg("lightsalmon")
        self.app.setSize("200x200")
        self.app.startSubWindow("Cat")
        self.app.addLabelEntry("Lat")
        self.app.addLabelEntry("Long")
        self.app.addButton("Update", on_btn_update_pressed)
        self.app.addButton('QUIT', self.app.stop)
        self.app.stopSubWindow()
    
    def test(self):
        print("starting")
    
    def send_data(self):
        cat= self.tracked_cat
        lat= self.lat
        long= self.long
        command= {"command":"send_data","name":cat, "lat":lat, "long":long}
        self.publish_command(command)
        
t0 = {'source': 'initial',
       'target': 'idle',
        'trigger':'you_there'}

t_start = {'trigger': 'start',
            'source': 'idle',
            'target': 'low_freq',
            'effect': 'test'}

t1 = {'trigger': 't',
       'source': 'low_freq',
       'target': 'low_freq',
       'effect': 'send_data'}

t2 = {'trigger': 'inside_fence',
       'source': 'low_freq',
       'target': 'norm_freq'}

t3 = {'trigger': 't',
       'source': 'norm_freq',
       'target': 'norm_freq',
       'effect': 'send_data'}

t4 = {'trigger': 'cat_in',
       'source': 'norm_freq',
       'target': 'low_freq'}

t5 = {'trigger': 'start_tracking',
       'source': 'norm_freq',
       'target': 'high_freq'}

t6 = {'trigger': 'outside_fence',
       'source': 'norm_freq',
       'target': 'high_freq'}

t7 = {'trigger': 't',
       'source': 'high_freq',
       'target': 'high_freq',
       'effect': 'send_data'}

t8 = {'trigger': 'cat_in',
       'source': 'high_freq',
       'target': 'low_freq'}

t9 = {'trigger': 'inside_fence',
       'source': 'high_freq',
       'target': 'norm_freq'}

idle = {'name': 'idle'}

low_freq = {'name': 'low_freq',
        'entry': 'start_timer("t", 10000)'}

norm_freq = {'name': 'norm_freq',
        'entry': 'start_timer("t", 5000)'}

high_freq = {'name': 'high_freq',
        'entry': 'start_timer("t", 1000)'}


debug_level = logging.DEBUG
logger = logging.getLogger(__name__)
logger.setLevel(debug_level)
ch = logging.StreamHandler()
ch.setLevel(debug_level)
formatter = logging.Formatter('%(asctime)s - %(name)-12s - %(levelname)-8s - %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

c = CollarComponent()
"""
stm_collar = Machine(name='stm_collar', transitions=[t0, t_start, t1, t2, t3, t4, t5, t6, t7, t8, t9], obj=c, states=[idle, low_freq, norm_freq, high_freq])
c.stm = stm_collar
driver = Driver()
driver.add_machine(stm_collar)
driver.start()
"""

c.app.go()

c.mqtt_client.loop_stop()
c.driver.stop()
